## Tarea NeuralWorks

### 1. Procesos automatizados para ingerir y almacenar los datos bajo demanda <br>
a. Los viajes que son similares en términos de origen, destino y hora del día deben agruparse. Describa el enfoque que utilizó para agregar viajes similares.

Para esta pregunta desarrollé una etl en Python que se encuentra en la ruta src/etl.py <br>
Usé SQlite3 como base de datos. En el archivo database/init.py se puede ver la estructura de las dos tablas que utilicé para la solución (Que explico más abajo)<br>
Para crear la base de datos se debe ejecutar el comando python3 init.py en la consola, dentro de la carpeta database.

In [74]:
import os

os.system("python3 ../database/init.py")

0

Asumí que el CSV con la data se deja en la carpeta data/trips.csv
<br>
Para almacenar viajes que son similares en términos de origen, destino y hora del día decidí aproximar las latitudes y longitudes a una cantidad menor de decimales. Esta cantidad se puede controlar con el parámetro de la línea 9. De esta forma, de la fila 36 a 47 me encargo de aproximar decimales y luego en la fila 54 hago la query del group by<br>
<br>
Algunos comentarios importantes que quiero hacer sobre mi código:<br>
1. Desarrollé el código de manera que se pueda ejecutar muchas veces (Es decir, se puede cargar un CSV, procesarlo, luego cargar otro y esa info se agregará a la anterior).
<br>
2. Al principio y al final del código utilizo una tabla de procesamiento de data donde voy dejando registro de cuándo se inició/terminó la ingesta. De esta manera, se puede saber si hay alguna ingesta de datos en curso o no con el servicio que detallo más abajo.
<br>
3. En la línea 86 a la 120 ejecuto tanto una query de inserción y otra de actualización de datos, donde además utilizo una tabla temporal. Esta no es la solución que me hubiese gustado, sin embargo SQLite3 no contempla UPSERT o el comando MERGE (como en SQL Server, por ejemplo), con el que podría haber hecho ambas funciones al mismo tiempo y, además, tampoco se puede updatear una tabla desde un SELECT, y por lo tanto tuve que usar una tabla temporal.<br>Por simplicidad utilicé SQLite3, pero el código queda menos elegante. Creo también que quizás el desafío no estaba hecho para procesar más de un CSV (algo que asumí desde el principio). En ese caso el código se simplificaría solamente con un insert.
4. La tabla de trips tiene también una columna que dice "min_datetime", con la cual almaceno la primera ocurrencia de ese registro. Esto lo hago porque, dado que desarrollé el código de manera que se pueda ejecutar muchas veces, para poder desarrollar el primer servicio de promedio de viajes semanales, necesitaba este dato para saber desde qué fecha se estaban considerando los datos y saber cuántas semanas habían pasado.

### 2. Un servicio que es capaz de proporcionar la siguiente funcionalidad:

Para desarrollar el servicio se creo una api en flask. El código está en src/api.py<br>
Para levantarla hay que ejecutar:<br>
export FLASK_APP=api.py<br>
flask run<br>
Tiene dos endpoints:<br>
1. /trips: Recibe la region y latitudes máximas y mínimas y retorna el promedio de cantidad de viajes semanales.<br> Para calcular el promedio semanal, primero filtro los datos con los parámetros anteriores, luego para cada registro tomo su min_datetime para saber hace cuántos días que se registran datos, para así saber cuántas semanas hay de registro de data. Luego sumo el promedio de todos los registros para sacar el promedio total.<br>
2. /data_status: Retorna el estado de ingesta de datos de la base. En caso de que se estén insertando datos retorna: "Data being ingested since" y la hora en la que se empezó a ingestar. En caso de que no, retorna "No data being ingested"
<br>
<br>

Probemos con comandos:

In [1]:
import requests
import json

In [42]:
api_url = "http://127.0.0.1:5000" #o la que entregue la consola al ejecutar flask run

In [76]:
trips_endpoint = "/trips"
trips_response = requests.get("{}{}".format(api_url,trips_endpoint), params={"region": "Turin", 
                                   "max_lat": 100.0, 
                                   "min_lat": -100.0,
                                  "max_lng": 100.0,
                                  "min_lng": -100.0})
print(trips_response.text)

{"avg_weekly_trips":0.057654169673924546,"status":"200"}



In [77]:
data_endpoint = "/data_status"
data_status_response = requests.get("{}{}".format(api_url,data_endpoint))
print(data_status_response.text)

{"Data":"No data being ingested","Status":"200"}



Ahora ejecuté la ETL agregándole un sleep de unos segundos para poder ejecutar la misma request y obtener otra respuesta y se obtiene lo siguiente:

In [79]:
data_endpoint = "/data_status"
data_status_response = requests.get("{}{}".format(api_url,data_endpoint))
print(data_status_response.text)

{"Data":"Data being ingested since 2023-01-12 22:25:01.964140","Status":"200"}



### 3.  La solución debe ser escalable a 100 millones de entradas. Se recomienda simplificar los datos mediante un modelo de datos. Agregue pruebas de que la solución es escalable

El código está desarrollado de manera que se lee el csv en chunks de data. (como se muestra en la línea 30). Por lo tanto el código está preparado para leer de a 1 millón de registros a la vez, para no saturar la memoria, por lo que debiese ser escalable a cualquier solución. Quizás si los chunks son muy pequeños el código tarda más, por lo que quizás habría que ir jugando con el tamaño.
<br>
También pensé en las siguientes opciones que no alcancé a implementar:<br>
1. Dentro de cada chunk procesar paralelamente cada región
2. El comando MERGE que comenté más arriba también habría acelerado el proceso de incersión

### 4. Puntos de bonificación si incluye su solución en contenedores y si dibuja cómo configuraría la aplicación en GCP

Nunca he utilizado GCP pero sí AWS, por lo que responderé con AWS:
1. Almacenaría los CSV para poder procesarlos en un bucket de S3. Además así queda registro histórico de la raw data que se va procsando
2. La ETL la desarrollaría en Glue, con un Job. Se podría utilizar Spark para mejorar el rendimiento y procesar más rápido que pandas, además de que ejecuta funciones de manera paralela.
3. La base de datos se puede montar en RDS
4. El servicio de API se puede desarrollar a través de API Gateway.